In [1]:
import os
print(os.getcwd())
import sys
sys.path.append('/local/scratch/kchan76/sparse-coarse-operator/')
from libs.pde import *
from libs.rotated_laplacian import RotatedLaplacian
from libs.models import BasicNet, MultiHeadedAttention
import torch

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print('device:', device)

/local/scratch/kchan76/sparse-coarse-operator/tests/laplacian/fixed-xi


/local/scratch/kchan76/sparse-stencil-learning/src/pyamg2/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device: cuda:1


# 3-Level Attention 

In [2]:
# seed = 63
list_eps = [1/200]
low_pi = 3
high_pi = 4
theta_low = np.pi/12*low_pi
theta_high = np.pi/12*high_pi
npts = 5
list_theta = np.linspace(theta_low, theta_high, npts)

In [3]:
train_grid_size = 31
k2 = 19
k3 = 15
same_vecs = False
PDE = RotatedLaplacian(train_grid_size,[k2,k3],list_eps, list_theta,'fixed xi',same_vecs=same_vecs)

In [4]:
# LEVEL 2
seed = 4
print('seed:', seed)
set_global_seed(seed)
h = 8
dhid = 256
epochs = 1000
adam_decay_rate = 0 # adam weight decay
lr = 1e-4 # learning rate
lr_decay_rate = 1 # learning rate decay
lr_decay_step = 1000 # learning rate decay step size
init = 'xavier uniform' # "<name> <distribution>" name={'xavier', 'kaiming'} distribution={'uniform', 'normal'}

ncopy = 1
single_model = False # KEEP IT FALSE!! if true, only model_prob will be learned
softmax_topk = False
dropout = 0.1

model_prob_l2 = MultiHeadedAttention(h ,dhid, ncopy=ncopy, init=init, dropout=dropout)
model_value_l2 = MultiHeadedAttention(h, dhid, ncopy=ncopy, init=init, dropout=dropout)


model_prob_l2,model_value_l2 = PDE.train(PDE.A2_train, 
                                         PDE.s2_train,
                                         PDE.eig_vec2_train,
                                         model_prob_l2,
                                         model_value_l2,
                                         epochs,
                                         adam_decay_rate,
                                         lr,
                                         lr_decay_rate,
                                         lr_decay_step,
                                         device,
                                         single_model=single_model,
                                         softmax_on = softmax_topk,
                                         verbose = True)




seed: 4
 epoch:  0  loss:  tensor(0.4813, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0157, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0111, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0111, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  999  loss:  tensor(0.0110, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)


## Level 3

In [5]:
# LEVEL 3
l3_seed = 62
print('seed:', l3_seed)
set_global_seed(l3_seed)
h3 = 8
dhid3 = 256
epochs3 = 1500
adam_decay_rate3 = 0 # adam weight decay
lr3 = 5e-5 # learning rate
lr_decay_rate3 = 0.8 # learning rate decay
lr_decay_step3 = 750 # learning rate decay step size
init3 = 'xavier uniform' 

ncopy3 = 1
single_model3 = False
softmax_topk3 = True
dropout3 = 0.5


model_prob_l3 = MultiHeadedAttention(h3 ,dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)
model_value_l3 = MultiHeadedAttention(h3, dhid3, ncopy=ncopy3, init=init3, dropout=dropout3)

model_prob_l3,model_value_l3 = PDE.train(PDE.A3_train, 
                                         PDE.s3_train,
                                         PDE.eig_vec3_train,
                                         model_prob_l3,
                                         model_value_l3,
                                         epochs3,
                                         adam_decay_rate3,
                                         lr3,
                                         lr_decay_rate3,
                                         lr_decay_step3,
                                         device,
                                         single_model= single_model3,
                                         softmax_on = softmax_topk3,
                                         verbose = True)

seed: 62
 epoch:  0  loss:  tensor(0.3600, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  250  loss:  tensor(0.0147, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  500  loss:  tensor(0.0039, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  750  loss:  tensor(0.0039, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1000  loss:  tensor(0.0039, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1250  loss:  tensor(0.0034, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)
 epoch:  1499  loss:  tensor(0.0034, device='cuda:1', dtype=torch.float64, grad_fn=<AddBackward0>)


In [7]:
test_seed = 15
# seed_list_127 =[]
# for test_seed in range(100):
set_global_seed(test_seed)
print('test seed:', test_seed)
sizes = [1023]
num_test = 10
models = {}
models['level2'] = model_prob_l2, model_value_l2
models['level3'] = model_prob_l3, model_value_l3
test_grid_size = 127
max_levels = 3
epsilon = tuple(list_eps)
theta = (theta_low, theta_high)
random_test = True
dropout_on = False
accel = 'gmres'
# accel = None


for test_grid_size in sizes:
    print('size:',test_grid_size)
    set_global_seed(test_seed)
    num_iter_standard, num_iter_learning, thetaList, _, _ = PDE.test_model(num_test,
                                                            models,
                                                            test_grid_size,
                                                            max_levels,
                                                            epsilon,
                                                            theta,
                                                            device,
                                                            top_accel = accel,
                                                            dropout_on=dropout_on,
                                                            random_test=random_test,
                                                            single_model=single_model,
                                                            softmax_on = softmax_topk,
                                                            verbose = True)
#     if np.mean(num_iter_learning) < np.mean(num_iter_standard):
#         seed_list_127.append(test_seed)
#         print("seed list:", seed_list_127)

test seed: 15
size: 1023
standard stencil iter:    11.7   standard stencil time:   35.81324124336243
learned stencil iter:    19.9   learned stencil time:   29.259760069847108


In [70]:
print(num_iter_standard)
print(num_iter_learning)

[104, 95, 92, 112, 109, 90, 103, 110, 103, 110]
[423, 414, 419, 576, 468, 424, 425, 464, 421, 497]
